## Notebook for bootstrapping evaluation

### Imports

In [18]:
import transformers
from datasets import load_metric
import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

### Parameters for number of iterations and number of selected items

In [19]:
num_samples = 200
sample_size = 1000

In [20]:
metric = load_metric("squad")

In [21]:
def compute_metrics_for_sample(sample):
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in zip(sample['id'], sample['prediction_text'])]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in sample]
    return metric.compute(predictions=formatted_predictions, references=references)

### Data loading

In [22]:
data = pd.read_json('valid_pred_labeled_with_added_from_func.json')

### Spliting of the data 

#### Based on distance of the closest word (from question) from the answer in context - items with distance lower or equal than 3 and items with distance higher than 3

In [23]:
data_distances = data[data.distances >= 0]
data_higher, data_lower = [x for _, x in data.groupby(data_distances['distances'] <= 3)]

### Computation of metrics for samples

In [24]:
exact_list_lower = []
f1_list_lower = []

for i in tqdm(range(num_samples)):
    df = data_lower.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_lower.append(metrics1['exact_match'])
    f1_list_lower.append(metrics1['f1'])

  0%|          | 0/200 [00:00<?, ?it/s]

In [25]:
d_lower = {'exact_match': exact_list_lower, 'f1': f1_list_lower}
d_lower
lower_than_4 = pd.DataFrame(d_lower)
lower_than_4

,exact_match,f1
0,84.0,90.846708
1,84.6,91.083183
2,83.3,89.857095
3,84.1,90.562758
4,84.4,90.841649
...,...,...
195,83.5,90.683836
196,83.1,90.170529
197,83.8,89.997452
198,84.0,90.788456


In [26]:
exact_list_higher = []
f1_list_higher = []

for i in tqdm(range(num_samples)):
    df = data_higher.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_higher.append(metrics1['exact_match'])
    f1_list_higher.append(metrics1['f1'])

  0%|          | 0/200 [00:00<?, ?it/s]

In [27]:
d_higher = {'exact_match': exact_list_higher, 'f1': f1_list_higher}
d_higher
higher_than_4 = pd.DataFrame(d_higher)
higher_than_4

,exact_match,f1
0,76.8,85.262767
1,76.2,85.448763
2,76.8,85.289513
3,75.3,84.098369
4,76.2,84.887286
...,...,...
195,75.4,84.798147
196,75.0,83.924188
197,75.0,85.048418
198,73.8,83.353506


### Quantiles

In [28]:
lower_than_4.quantile([0.025, 0.975])

,exact_match,f1
0.025,81.5000,89.203266
0.975,85.3025,91.658113


In [29]:
higher_than_4.quantile([0.025, 0.975])

,exact_match,f1
0.025,74.2875,83.591081
0.975,78.4025,86.670196


In [30]:
lower_than_4.describe()

,exact_match,f1
count,200.00000,200.000000
mean,83.55750,90.499838
std,0.95151,0.658900
min,80.60000,87.897474
25%,83.00000,90.086818
50%,83.60000,90.513260
75%,84.22500,90.958312
max,85.60000,92.109433


In [31]:
higher_than_4.describe()

,exact_match,f1
count,200.00000,200.000000
mean,76.22800,85.123828
std,1.06068,0.824414
min,73.10000,82.740850
25%,75.50000,84.481497
50%,76.20000,85.142433
75%,76.90000,85.704912
max,78.90000,86.942339


### Spliting of the data

#### Based on count of the similar words between question and context - lower or equal to 4 and higher than 4

In [88]:
data_similar_words_higher, data_similar_words_lower = [x for _, x in data.groupby(data['similar_words'] <= 4)]
print('Higher count data len: ', len(data_similar_words_higher))
print('Lower count data len: ', len(data_similar_words_lower))

Higher count data len:  5740
Lower count data len:  4830


### Computation of metrics for samples

In [89]:
exact_list_similar_words_lower = []
f1_list_similar_words_lower = []

for i in tqdm(range(num_samples)):
    df = data_similar_words_lower.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_similar_words_lower.append(metrics1['exact_match'])
    f1_list_similar_words_lower.append(metrics1['f1'])

  0%|          | 0/200 [00:00<?, ?it/s]

In [90]:
d_similar_words_lower = {'exact_match': exact_list_similar_words_lower, 'f1': f1_list_similar_words_lower}
d_similar_words_lower
lower_similar_words_than_4 = pd.DataFrame(d_similar_words_lower)
lower_similar_words_than_4

,exact_match,f1
0,77.9,86.201630
1,75.6,84.616505
2,75.7,84.998308
3,76.6,85.272872
4,77.3,85.852570
...,...,...
195,78.6,87.390954
196,77.3,86.016487
197,77.7,86.395169
198,77.7,85.788061


In [91]:
exact_list_similar_words_higher = []
f1_list_similar_words_higher = []

for i in tqdm(range(num_samples)):
    df = data_similar_words_higher.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_similar_words_higher.append(metrics1['exact_match'])
    f1_list_similar_words_higher.append(metrics1['f1'])

  0%|          | 0/200 [00:00<?, ?it/s]

In [92]:
d_similar_words_higher = {'exact_match': exact_list_similar_words_higher, 'f1': f1_list_similar_words_higher}
d_similar_words_higher
higher_similar_words_than_4 = pd.DataFrame(d_similar_words_higher)
higher_similar_words_than_4

,exact_match,f1
0,82.9,89.668875
1,82.0,89.146711
2,85.1,91.292489
3,82.1,89.113918
4,82.2,88.752351
...,...,...
195,82.3,89.376262
196,82.3,89.489812
197,84.4,90.674647
198,81.8,88.256092


### Quantiles

In [93]:
lower_similar_words_than_4.quantile([0.025, 0.975])

,exact_match,f1
0.025,75.4000,84.693802
0.975,79.9025,87.822978


In [94]:
higher_similar_words_than_4.quantile([0.025, 0.975])

,exact_match,f1
0.025,80.5975,87.976679
0.975,84.8050,91.222243


In [95]:
lower_similar_words_than_4.describe()

,exact_match,f1
count,200.000000,200.000000
mean,77.751500,86.170985
std,1.185845,0.800690
min,75.000000,84.148025
25%,76.800000,85.648972
50%,77.850000,86.092368
75%,78.600000,86.687110
max,80.900000,88.708345


In [96]:
higher_similar_words_than_4.describe()

,exact_match,f1
count,200.00000,200.000000
mean,82.67850,89.498549
std,1.15471,0.842324
min,79.30000,87.639278
25%,81.90000,88.921644
50%,82.70000,89.436036
75%,83.42500,89.917512
max,85.70000,91.702404
